In [157]:
import math
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib import pyplot as plt

In [158]:
# Enter here location to your test data and modell
activelosses = "../results_group2/Active-Losses-2022_test.csv"
renewablegen = "../results_group2/Forecast-renewable-generation_2022_test.csv"
forecasttemp = '../results_group2/eq_temp_2022_test.csv.csv'
ntc = '../results_group2/ntc_2022_test.csv'
#modell = "modell_group_02_v01.json"

In [159]:
activeslosses = pd.read_csv(activelosses)
activeslosses.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
activeslosses = activeslosses.iloc[:, 2:]
# Rename column 'A' to 'X'
activeslosses['datetime'] = pd.to_datetime(activeslosses['datetime'])
activeslosses.rename(columns={'Wirkverluste/Active Losses': 'MWh'}, inplace=True)
activeslosses.set_index(activeslosses.columns[0], inplace=True)
activeslosses = activeslosses[~activeslosses.index.duplicated(keep='first')]
activeslosses

,MWh
datetime,
2022-01-01 00:00:00,146.054792
2022-01-01 01:00:00,139.133354
2022-01-01 02:00:00,147.562500
2022-01-01 03:00:00,157.636204
2022-01-01 04:00:00,163.326766
...,...
2022-12-31 19:00:00,67.876028
2022-12-31 20:00:00,72.765318
2022-12-31 21:00:00,81.277633


In [160]:
Forecast_renew = pd.read_csv(renewablegen, skiprows=0)
Forecast_renew = Forecast_renew.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
# Load Forecast Renewable Generation data
Forecast_renew['datetime'] = pd.to_datetime(Forecast_renew['datetime'])
Forecast_renew.set_index(Forecast_renew.columns[0], inplace=True)
Forecast_renew = Forecast_renew[~Forecast_renew.index.duplicated(keep='first')]
Forecast_renew = Forecast_renew.resample('H').asfreq()
Forecast_renew

,solar_fore_de_mw,solar_fore_it_mw,wind_fore_de_mw,wind_fore_it_mw
datetime,,,,
2022-01-01 00:00:00,0.0,0.0,31805.65,1331.48
2022-01-01 01:00:00,0.0,0.0,29880.67,1438.15
2022-01-01 02:00:00,0.0,0.0,28826.75,1623.80
2022-01-01 03:00:00,0.0,0.0,27631.75,1894.75
2022-01-01 04:00:00,0.0,0.0,27128.00,2335.05
...,...,...,...,...
2022-12-31 19:00:00,0.0,0.0,44107.32,569.68
2022-12-31 20:00:00,0.0,0.0,44512.60,459.45
2022-12-31 21:00:00,0.0,0.0,44946.45,399.35


In [161]:
Forecast_temp = pd.read_csv(forecasttemp, skiprows=0)
Forecast_temp = Forecast_temp.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
Forecast_temp['datetime'] = pd.to_datetime(Forecast_temp['datetime'])
Forecast_temp.set_index(Forecast_temp.columns[0], inplace=True)
Forecast_temp = Forecast_temp[~Forecast_temp.index.duplicated(keep='first')]
Forecast_temp



,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
datetime,,,,
2022-01-01 00:00:00,6.65,7.90,10.25,7.38
2022-01-01 01:00:00,6.77,8.31,10.64,7.25
2022-01-01 02:00:00,6.42,8.01,10.46,7.09
2022-01-01 03:00:00,6.08,7.77,10.21,6.94
2022-01-01 04:00:00,5.68,7.62,10.07,6.85
...,...,...,...,...
2022-12-31 19:00:00,10.37,14.18,14.13,11.68
2022-12-31 20:00:00,9.67,13.90,13.91,10.76
2022-12-31 21:00:00,9.10,13.57,13.79,10.06


In [162]:
# Load NTC data
NTC = pd.read_csv(ntc, skiprows=0)
NTC = NTC.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
NTC['datetime'] = pd.to_datetime(NTC['datetime'])
NTC.set_index(NTC.columns[0], inplace=True)
NTC = NTC[~NTC.index.duplicated(keep='first')]
NTC = NTC.resample('H').asfreq()
NTC = NTC.interpolate(method='polynomial', order=2)
NTC

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
datetime,,,,,,,,
2022-01-01 00:00:00,1200.0,4000.0,1400.0,3158.0,1200.0,800.0,3200.0,1910.0
2022-01-01 01:00:00,1200.0,4000.0,1400.0,3213.0,1200.0,800.0,3200.0,1910.0
2022-01-01 02:00:00,1200.0,4000.0,1400.0,2824.0,1200.0,800.0,3200.0,1910.0
2022-01-01 03:00:00,1200.0,4000.0,1400.0,2678.0,1200.0,800.0,3200.0,1910.0
2022-01-01 04:00:00,1200.0,4000.0,1400.0,2629.0,1200.0,800.0,3200.0,1910.0
...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
2022-12-31 20:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
2022-12-31 21:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0


In [168]:
# Data preprocessing steps
activeslosses = activeslosses.interpolate(method='polynomial', order=2)
activeslosses_hour = activeslosses.resample('H').sum()
Forecast_renew = Forecast_renew.interpolate(method='polynomial', order=2)
Forecast_temp_hourly = Forecast_temp.resample('H').asfreq()
Forecast_temp_hourly_lin = Forecast_temp_hourly.interpolate(method='linear')
Forecast_temp_hourly_poly = Forecast_temp_hourly.interpolate(method='polynomial', order=1)
NTC = NTC.interpolate(method='polynomial', order=2)

In [169]:
# Merge the dataframes
merged_df = pd.merge(activeslosses, Forecast_renew, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, Forecast_temp, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, NTC, left_index=True, right_index=True, how='outer')

In [170]:
merged_df = merged_df.reset_index()

In [171]:
merged_df

,datetime,MWh,solar_fore_de_mw,solar_fore_it_mw,wind_fore_de_mw,wind_fore_it_mw,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
0,2022-01-01 00:00:00,146.054792,0.0,0.0,31805.65,1331.48,6.65,7.90,10.25,7.38,1200.0,4000.0,1400.0,3158.0,1200.0,800.0,3200.0,1910.0
1,2022-01-01 01:00:00,139.133354,0.0,0.0,29880.67,1438.15,6.77,8.31,10.64,7.25,1200.0,4000.0,1400.0,3213.0,1200.0,800.0,3200.0,1910.0
2,2022-01-01 02:00:00,147.562500,0.0,0.0,28826.75,1623.80,6.42,8.01,10.46,7.09,1200.0,4000.0,1400.0,2824.0,1200.0,800.0,3200.0,1910.0
3,2022-01-01 03:00:00,157.636204,0.0,0.0,27631.75,1894.75,6.08,7.77,10.21,6.94,1200.0,4000.0,1400.0,2678.0,1200.0,800.0,3200.0,1910.0
4,2022-01-01 04:00:00,163.326766,0.0,0.0,27128.00,2335.05,5.68,7.62,10.07,6.85,1200.0,4000.0,1400.0,2629.0,1200.0,800.0,3200.0,1910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31 19:00:00,67.876028,0.0,0.0,44107.32,569.68,10.37,14.18,14.13,11.68,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
8756,2022-12-31 20:00:00,72.765318,0.0,0.0,44512.60,459.45,9.67,13.90,13.91,10.76,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
8757,2022-12-31 21:00:00,81.277633,0.0,0.0,44946.45,399.35,9.10,13.57,13.79,10.06,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
8758,2022-12-31 22:00:00,95.496046,0.0,0.0,44938.83,420.78,8.64,13.29,13.64,9.61,1200.0,4000.0,1200.0,1459.0,1000.0,800.0,3700.0,1810.0


In [122]:
# Preprocess data

In [123]:
def preprocess_and_merge_data(activelosses1, renewablegen1, forecasttemp1, ntc1):
    # Load Active Losses data
    activeslosses = pd.read_csv(activelosses1)
    activeslosses = activeslosses.iloc[:, 2:]
    activeslosses['datetime'] = pd.to_datetime(activeslosses['Zeitstempel']) - pd.Timedelta(minutes=15)
    activeslosses.set_index(activeslosses.columns[0], inplace=True)
    activeslosses['MWh'] = activeslosses['Wirkverluste/Active Losses'] / 1000
    activeslosses = activeslosses[~activeslosses.index.duplicated(keep='first')]
    activeslosses = activeslosses.resample('15T').asfreq()

    # Load Forecast Renewable Generation data
    Forecast_renew = pd.read_csv(renewablegen1, skiprows=0)
    Forecast_renew['datetime'] = pd.to_datetime(Forecast_renew['datetime'])
    Forecast_renew.set_index(Forecast_renew.columns[0], inplace=True)
    Forecast_renew = Forecast_renew[~Forecast_renew.index.duplicated(keep='first')]
    Forecast_renew = Forecast_renew.resample('H').asfreq()

    # Load Forecast Temperature data
    Forecast_temp = pd.read_csv(forecasttemp1, skiprows=0)
    Forecast_temp['datetime'] = pd.to_datetime(Forecast_temp['datetime'])
    Forecast_temp.set_index(Forecast_temp.columns[0], inplace=True)
    Forecast_temp = Forecast_temp[~Forecast_temp.index.duplicated(keep='first')]

    # Load NTC data
    NTC = pd.read_csv(ntc1, skiprows=0)
    NTC['datetime'] = pd.to_datetime(NTC['datetime'])
    NTC.set_index(NTC.columns[0], inplace=True)
    NTC = NTC[~NTC.index.duplicated(keep='first')]
    NTC = NTC.resample('H').asfreq()

    # Data preprocessing steps
    activeslosses = activeslosses.interpolate(method='polynomial', order=2)
    activeslosses_hour = activeslosses.resample('H').sum()
    Forecast_renew = Forecast_renew.interpolate(method='polynomial', order=2)
    Forecast_temp_hourly = Forecast_temp.resample('H').asfreq()
    Forecast_temp_hourly_lin = Forecast_temp_hourly.interpolate(method='linear')
    Forecast_temp_hourly_poly = Forecast_temp_hourly.interpolate(method='polynomial', order=1)
    Forecast_temp_hourly_poly.loc[pd.to_datetime('2019-01-01 00:00:00')] = Forecast_temp_hourly_poly.loc['2019-01-01 01:00:00']
    NTC = NTC.interpolate(method='polynomial', order=2)

    # Merge the dataframes
    merged_df = pd.merge(activeslosses_hour, Forecast_renew, left_index=True, right_index=True, how='outer')
    merged_df = pd.merge(merged_df, Forecast_temp_hourly_poly, left_index=True, right_index=True, how='outer')
    merged_df = pd.merge(merged_df, NTC, left_index=True, right_index=True, how='outer')
    
    # Save the merged dataframe to a CSV file
    #merged_df.to_csv('merged_data.csv', index=True)
    
    return merged_df.reset_index()
    
    

# Call the function to preprocess and merge the data
df = preprocess_and_merge_data(activelosses, renewablegen, forecasttemp, ntc)
display(df)

KeyError: 'Zeitstempel'

In [172]:
df = merged_df

In [173]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.head()

,datetime,MWh,solar_fore_de_mw,solar_fore_it_mw,wind_fore_de_mw,wind_fore_it_mw,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
0,2022-01-01 00:00:00,146.054792,0.0,0.0,31805.65,1331.48,6.65,7.90,10.25,7.38,1200.0,4000.0,1400.0,3158.0,1200.0,800.0,3200.0,1910.0
1,2022-01-01 01:00:00,139.133354,0.0,0.0,29880.67,1438.15,6.77,8.31,10.64,7.25,1200.0,4000.0,1400.0,3213.0,1200.0,800.0,3200.0,1910.0
2,2022-01-01 02:00:00,147.562500,0.0,0.0,28826.75,1623.80,6.42,8.01,10.46,7.09,1200.0,4000.0,1400.0,2824.0,1200.0,800.0,3200.0,1910.0
3,2022-01-01 03:00:00,157.636204,0.0,0.0,27631.75,1894.75,6.08,7.77,10.21,6.94,1200.0,4000.0,1400.0,2678.0,1200.0,800.0,3200.0,1910.0
4,2022-01-01 04:00:00,163.326766,0.0,0.0,27128.00,2335.05,5.68,7.62,10.07,6.85,1200.0,4000.0,1400.0,2629.0,1200.0,800.0,3200.0,1910.0


In [174]:
df.isna().sum()

datetime                0
MWh                     0
solar_fore_de_mw        0
solar_fore_it_mw        0
wind_fore_de_mw         0
wind_fore_it_mw         0
temperature_fore_ch    19
temperature_fore_fr     1
temperature_fore_de     1
temperature_fore_it     1
CH_AT                   0
CH_DE                   0
CH_FR                   0
CH_IT                   0
AT_CH                   0
DE_CH                   0
FR_CH                   0
IT_CH                   0
dtype: int64

In [182]:
# Remove rows containing any NaN values
df = df.dropna()

In [183]:
dat = df.copy()
# Extract day of the week (0 = Monday, 6 = Sunday)
dat['weekday'] = dat['datetime'].dt.weekday
dat['month'] = dat['datetime'].dt.month
dat['time'] = dat['datetime'].dt.time

# Encode 'month' and 'weekday' as categorical variables
dat['month'] = dat['month'].astype('category')
dat['weekday'] = dat['weekday'].astype('category')

# Perform one-hot encoding on 'month' and 'weekday'
df_encoded = pd.get_dummies(dat, columns=['month', 'weekday', "time"], prefix=['month', 'weekday', 'time'])

# drop the datetime variable
df_encoded = df_encoded.drop(["datetime"], axis=1)

In [184]:
import numpy as np

data = df_encoded

# Define sequence lengths
input_seq_length = 7 * 24  # Previous 7 days
output_seq_length = 24     # Next 24 hours

# Initialize lists to store sequences
input_sequences = []
output_sequences = []

# Iterate over the data to create sequences
for i in range(0, len(data) - input_seq_length - output_seq_length + 1, output_seq_length):
    input_seq = data.iloc[i : i + input_seq_length].drop(columns=['MWh'])
    output_seq = data.iloc[i + input_seq_length : i + input_seq_length + output_seq_length]['MWh'].values
    
    # Append the sequences to the lists
    input_sequences.append(input_seq.values)
    output_sequences.append(output_seq)

# Convert lists to NumPy arrays
input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

# 'input_sequences' now contains input sequences with shape (num_samples, input_seq_length, num_features)
# 'output_sequences' now contains corresponding output sequences with shape (num_samples, output_seq_length)


In [185]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch.nn as nn



In [191]:

# Ensure the data types are compatible with PyTorch tensors
input_test = input_sequences.astype(np.float32)
output_test = output_sequences.astype(np.float32)

# Convert NumPy arrays to PyTorch tensors
input_test_tensor = torch.tensor(input_test, dtype=torch.float32)
output_test_tensor = torch.tensor(output_test, dtype=torch.float32)

# Create PyTorch datasets
test_dataset = TensorDataset(input_test_tensor, output_test_tensor)

# Define batch size
batch_size = 64  # Adjust as needed

# Create PyTorch dataloaders
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [192]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batchnorm = nn.BatchNorm1d(hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.05)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # Apply batch normalization
        out = self.batchnorm(out.transpose(1, 2)).transpose(1, 2)  # Transpose for correct dimension
        return out, (hn, cn)
    
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=0.05)
        self.batchnorm = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hn, cn):
        out, (hn, cn) = self.lstm(x, (hn, cn))
        # Apply batch normalization
        out = self.batchnorm(out.transpose(1, 2)).transpose(1, 2)  # Transpose for correct dimension
        out = self.fc(out[:, -1, :])  # Use the last time step's output
        return out

class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(hidden_size, output_size, num_layers)

    def forward(self, x):
        encoder_out, (hn, cn) = self.encoder(x)
        decoder_out = self.decoder(encoder_out, hn, cn)
        return decoder_out


In [197]:
import pickle

# Save the model to a file using pickle
model_path = "group2_seq2seq V3 full.pkl" # Replace with your desired file path
    
# Load the model from the pickle file
with open(model_path, 'rb') as model_file:
    model = pickle.load(model_file)

In [198]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.eval()
total_mae = 0.0  # Initialize the total mean absolute error
all_predictions = []  # Initialize a list to store all predictions
all_y_test = []

with torch.no_grad():
    for batch_input, batch_target in test_loader:
        # Forward pass
        batch_input = batch_input.to(device)
        batch_target = batch_target.to(device)
        
        predictions = model(batch_input)
        
        # Append the predictions to the list
        all_predictions.append(predictions.cpu().numpy())  # Assuming predictions is a PyTorch tensor
        all_y_test.append(batch_target.cpu().numpy()) 
        
        # Calculate the Mean Absolute Error (MAE)
        mae = torch.mean(torch.abs(predictions - batch_target), )

        print(mae)
        # Accumulate the MAE for each batch
        total_mae += mae.item()

# Calculate the average MAE for the entire test set
average_mae = total_mae / len(test_loader)

print(f'Average MAE on Test Set: {average_mae:.4f}')


# Concatenate predictions from all batches into a single numpy array
all_predictions = np.concatenate(all_predictions, axis=0)

tensor(37.8398, device='cuda:0')
tensor(34.3180, device='cuda:0')
tensor(21.8778, device='cuda:0')
tensor(29.9601, device='cuda:0')
tensor(41.9021, device='cuda:0')
tensor(29.3537, device='cuda:0')
Average MAE on Test Set: 32.5419
